# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

In [1]:
! pip install transformers datasets evaluate

In [18]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import PushToHubCallback
from transformers import pipeline

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [3]:
dataset = load_dataset("CUTD/math_df", split="train[:50]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 50
})

### split the train into trainin and test set with 20%

In [5]:
dataset = dataset.train_test_split(test_size=0.2)

### take a look at an example:

In [6]:
dataset["train"][0]

{'Unnamed: 0': 3,
 'text': 'A pragmatic politician always considering the budget and community opinions before approving projects'}

In [7]:
dataset["test"][0]

{'Unnamed: 0': 41,
 'text': 'A seasoned TCG player and avid Marvel comics fan who is always on the lookout for new games and enjoys analyzing game mechanics and developer pedigrees.'}

## Step 2: Load the Pretrained Model and Tokenizer

Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModelForMaskedLM.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


## Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [9]:
def preprocess(examples):
    examples['text'] = [text.lower() for text in examples['text']]
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    return inputs

In [10]:
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Step 4: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

In [12]:
tf_train_set = model.prepare_tf_dataset(
    dataset["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    dataset["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [13]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

## Step 5: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

In [15]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
5/5 [==============================] - 480s 96s/step - loss: 3.0862 - val_loss: 3.9184
Epoch 2/3
5/5 [==============================] - 377s 74s/step - loss: 3.0558 - val_loss: 2.1427
Epoch 3/3
5/5 [==============================] - 372s 76s/step - loss: 2.2698 - val_loss: 1.2341


In [16]:
model.save_pretrained("my_mlm_model")
tokenizer.save_pretrained("my_toknizer")

('my_toknizer/tokenizer_config.json',
 'my_toknizer/special_tokens_map.json',
 'my_toknizer/vocab.txt',
 'my_toknizer/added_tokens.json',
 'my_toknizer/tokenizer.json')

## Step 6: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

## Step 7: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [30]:
mask_filler = pipeline("fill-mask", model="my_mlm_model", tokenizer = "my_toknizer")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at my_mlm_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [31]:
sen = "The capital of France is [MASK]."

predictions = mask_filler(sen)

In [32]:
predictions

[{'score': 0.37879326939582825,
  'token': 3000,
  'token_str': 'paris',
  'sequence': 'the capital of france is paris.'},
 {'score': 0.07682731747627258,
  'token': 22479,
  'token_str': 'lille',
  'sequence': 'the capital of france is lille.'},
 {'score': 0.049827512353658676,
  'token': 10241,
  'token_str': 'lyon',
  'sequence': 'the capital of france is lyon.'},
 {'score': 0.04241811856627464,
  'token': 16766,
  'token_str': 'marseille',
  'sequence': 'the capital of france is marseille.'},
 {'score': 0.0341377891600132,
  'token': 17209,
  'token_str': 'toulouse',
  'sequence': 'the capital of france is toulouse.'}]